# III. Precomputing your favorite PBH accretion emission model (WIP)

In 2509.22772, we explored a set of PBH accretion models as variations to base models $\textbf{PR}$ and $\textbf{BHL}$, based on Park and Ricotti (2013) and the Bondi-Hoyle-Lyttleton accretion model respectively, with the emission mechanism based on the hot Advection-Dominated Accretion Flow (ADAF) model. We have demonstrated that model variations to both the accretion and the emission model can impact the modification to the $T_{21}$ power spectrum, and hence the projected sensitivity for HERA. Here, we provide the reader with a guide to run `DM21cm` with alternative modeling choices to further explore the modeling variations.

# 1. Modify existing `PBHAccretionModel` options

In [1]:
%reload_ext autoreload
%autoreload 2

from dm21cm.precompute.accretion import PBHAccretionModel

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/halomod/halo_model.py:32: UserWarning: Warning: Some Halo-Exclusion models have significant speedup when using Numba
  from .halo_exclusion import Exclusion, NoExclusion


The options for `PBHAccretionModel` can be viewed in the docstring:

In [2]:
PBHAccretionModel?

Init signature:
PBHAccretionModel(
    model,
    c_in=23,
    lambda_fudge=1,
    v_rel_type='DMRest',
    delta_e=0.1,
)
Docstring:      <no docstring>
Init docstring:
PBH accretion emission model.

Args:
    model (str): supports 'PR-ADAF', 'BHL-ADAF', 'PRHALO-ADAF'.
    c_in (float): Sound speed inside the I-front [km/s] in PR models. Required for PR models.
    lambda_fudge (float): Fudge factor in front of Mdot. Required for BHL models.
    v_rel_type {'DMDM', 'DMRest'}: Velocity distribution option.
        'DMDM'   - DM-DM relative velocity distribution function.
        'DMRest' - DM-rest frame relative velocity distribution function (default).
    delta_e (float): ADAF electron heating fraction. Default is 0.1.
File:           ~/dm21cm/DM21cm/src/dm21cm/precompute/accretion.py
Type:           type
Subclasses:     

For example, we can combine the effect of a decreased interior sound speed $c_\text{in}$, and an increased electron heating fraction $\delta_e$ (i.e. combining $\textbf{PR}$-$c_\text{in}^-$ and $\textbf{PR}$-$\delta_e^+$, see text in Sec. III of 2509.22772), by doing:

In [3]:
am = PBHAccretionModel(model='PR-ADAF', c_in=14, delta_e=0.5)

One can then put this line in the `===== Initialization =====` section of [build_pbhacc_tables.py](../src/dm21cm/precompute/scripts/build_pbhacc_tables.py), and run the script.

# 2. Defining a fully custom model